## 공학설계 과제 1

본 과제는 4주차 수업시간 중 진행한 실습의 연장선에서 2차원 convolution에 대한 이론을 학습하고, 다른 종류의 kernel 함수를 적용하는 사례를 학습함.

아래의 각 문항 별로 본 문서 내에 주어진 답변용 code 또는 markdown 셀에 답변을 작성하면 됨. 
과제 답변 작성 중 인터넷 검색으로 도출된 참고자료를 활용하여 과제를 수행한 경우, 반드시 참고한 자료의 출처를 명시해야 함.
ChatGPT를 활용한 경우에도 활용 여부를 꼭 명시하기 바람.
본 과제에서 아래 패키지들은 기본적으로 활용되므로, 해당 패키지 웹사이트의 API reference는 참고자료 출처 명시할 필요 없음
* Numpy: https://numpy.org/
* Scikit-image: https://scikit-image.org/
* MatPlotLib: https://matplotlib.org/

출처 명시 방법:
    코드 셀 내에 주석으로 또는 별도의 markdown 셀에
        저자/기관, 문서/페이지 제목, 출판지/웹사이트 제목, 날짜, url 등 순으로 기재함
출처 명시 방법 예시:
    # OpenCV.org, "Smoothing Images," OpenCV documentation, 2021.09.27 조회함, 
    # url: https://docs.opencv.org/master/dc/dd3/tutorial_gausian_median_blur_bilateral_filter.html


##### 과제 수행 기간: 10.03(화)~10.10(화)

##### 제출 마감: 10.10(화) 15:00

### 아래 셀들은 4주차 수업 시간에 학습한 내용임

##### 환경설정:
ipynb 파일에서는 셀을 실행시킨 순서대로 메모리에 저장하므로 아래 셀을 최초에 실행하여 패키지들을 import하고 실험용 영상을 읽어온 상태로 만들어서 나머지 셀을 수행하도록 함

In [ ]:
# 패키지 import
import numpy as np
import skimage
from skimage import io
from matplotlib import pyplot as plt

##### 영상 loading:
아래 셀은 convolution을 적용하고자 하는 영상을 불러오고, 해당 영상을 정수형 영상(각 픽셀 값이 [0~255] 범위의 값을 가짐)에서 소수점형 영상(픽셀 값이 [0~1] 범위의 값을 가짐)으로 변형하는 과정을 수행함

In [ ]:
img = io.imread("dog.jpg") #파일 경로는 각자의 환경 및 파일에 맞춰서 수정하여 실행함
img = img/255.0

# 읽어온 영상을 확인하고 싶으면 아래 줄 주석 해제하여 실행함
# plt.imshow(img)
# print(img.shape)

##### 영상 컬러 -> 흑백 변환:

읽어들인 컬러 영상을 흑백으로 변환함. 이때 밝기값을 픽셀의 R,G,B 각 값의 평균으로 정의함. 

NumPy의 slicing 기능을 통해 R,G,B의 각 채널을 행렬로 조회하여 vectorization 방식의 연산을 수행함

In [ ]:
img_bw = np.zeros((img.shape[0],img.shape[1]))
img_bw[:,:] = (img[:,:,0]+img[:,:,1]+img[:,:,2])/3
# 변환한 영상을 확인하고 싶으면 아래 줄 주석 해제하여 실행함
# plt.imshow(img_bw, cmap='gray')

##### 2차원 box filter 

신호(영상) x에 convolution을 적용하는 함수 h를 kernel이라고 부를 때, 단위 영역에 대해 상수 값을 갖는 kernel과의 convolution 과정을 box filtering이라고 지칭함. 이때 h의 크기가 만약 3x3이면 h[i,j] = 1/9 = 1/(3*3)으로 정의됨.

Box filtering을 통해 x의 이동 평균(moving average)이 계산됨.

아래 두 셀은 각각 1x3과 3x3 크기를 갖는 kernel과의 box filtering 연산을 수행하는 코드임.

In [ ]:
### 1x3 box filtering
img_conv = img_bw.copy()
H = img.shape[0]
W = img.shape[1]
kernel = np.array([1,1,1])
num_repeat = 100 # filtering을 반복하는 횟수 <- filtering 전/후 효과를 더 극명하게 하기 위해 (결과 확인용)
for k in range(num_repeat):
    img_conv[1:H-1,1:W-1] = (kernel[0]*img_conv[1:H-1,0:W-2]+
                             kernel[1]*img_conv[1:H-1,1:W-1]+
                             kernel[2]*img_conv[1:H-1,2:W])/3
# 변환한 영상을 확인하고 싶으면 아래 줄 주석 해제하여 실행함
# plt.imshow(img_conv, cmap='gray')

In [ ]:
### 3x3 box filtering
img_conv = img_bw.copy()
H = img.shape[0]
W = img.shape[1]
# kernel = np.array([1,1,1])
kernel = np.array([[1,1,1],[1,1,1],[1,1,1]])
num_repeat = 100 # filtering을 반복하는 횟수 <- filtering 전/후 효과를 더 극명하게 하기 위해 (결과 확인용)
for k in range(num_repeat):
    img_conv[1:H-1,1:W-1] = (kernel[0,0]*img_conv[0:H-2,0:W-2]+
                             kernel[0,1]*img_conv[0:H-2,1:W-1]+
                             kernel[0,2]*img_conv[0:H-2,2:W]+
                             kernel[1,0]*img_conv[1:H-1,0:W-2]+
                             kernel[1,1]*img_conv[1:H-1,1:W-1]+
                             kernel[1,2]*img_conv[1:H-1,2:W]+                             
                             kernel[2,0]*img_conv[2:H,0:W-2]+
                             kernel[2,1]*img_conv[2:H,1:W-1]+
                             kernel[2,2]*img_conv[2:H,2:W])/9
# 변환한 영상을 확인하고 싶으면 아래 줄 주석 해제하여 실행함
# plt.imshow(img_conv, cmap='gray')

##### 문제 1: Box filtering 이론 학습

Box filtering과 관련된 이론과 연관되는 내용을 우리 수업 교재에서 찾아서 확인하고 해당 내용을 간단하게 요약하여 작성하시오.

##### 문제 1 답변

-여기에 답변을 작성하시오. 답변 작성시 본 문구는 삭제하고 작성하시오.-

##### 문제 2: 5x5 Box filtering 코드 작성

아래 셀에 5x5 크기를 갖는 kernel과의 box filtering 연산을 수행하는 코드를 작성하시오.

In [ ]:
### 5x5 box filtering

# YOUR CODE HERE #

##### 문제 3: 7x7 Box filtering 코드 작성

아래 셀에 7x7 크기를 갖는 kernel과의 box filtering 연산을 수행하는 코드를 작성하시오.

In [ ]:
### 7x7 box filtering

# YOUR CODE HERE #

##### 문제 4: NxN Box filtering 코드 작성

아래 셀에 임의의 NxN 크기를 갖는 kernel과의 box filtering 연산을 수행하는 코드를 작성하시오.

Hint: 3x3, 5x5, 7x7 등 크기의 kernel들과의 연산 코드에서 [반복문]을 이용할 수 있는 여부를 고려하여 작성하시오.

In [ ]:
### NxN box filtering

# YOUR CODE HERE #

##### 문제 5-1: Sobel filtering 코드 작성 - X 방향

Kernel의 값을 다르게 하면 완전히 다른 효과를 갖는 연산이 됨. 
```
kernel = [[-1,0,1],[-2,0,2],[-1,0,1]]
```

으로 정의되는 convolution 연산, 즉 가로 방향 sobel filter을 수행하는 코드를 작성하시오.

In [ ]:
### Sobel filtering - X방향

# YOUR CODE HERE #

##### 문제 5-2: Sobel filtering 코드 작성 - Y 방향

Kernel의 값을 다르게 하면 완전히 다른 효과를 갖는 연산이 됨. 
```
kernel = [[1,2,1],[0,0,0],[-1,-2,-1]]
```

으로 정의되는 convolution 연산, 즉 세로 방향 sobel filter을 수행하는 코드를 작성하시오.

In [ ]:
### Sobel filtering - Y방향

# YOUR CODE HERE #

##### 문제 6: Sobel filtering 이론 학습

Sobel filtering과 관련된 이론과 연관되는 내용을 우리 수업 교재에서 찾아서 확인하고 해당 내용을 간단하게 요약하여 작성하시오.

##### 문제 6 답변

-여기에 답변을 작성하시오. 답변 작성시 본 문구는 삭제하고 작성하시오.-